In [11]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from PIL import Image
from tqdm import tqdm

In [12]:
# -------------------------------------------------------------
# Define Device
# -------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -------------------------------------------------------------
# Load Pretrained EfficientNet-B0 with 1024-D Output
# -------------------------------------------------------------
weights = EfficientNet_B0_Weights.IMAGENET1K_V1
base_model = efficientnet_b0(weights=weights)

# Freeze parameters (optional)
for param in base_model.parameters():
    param.requires_grad = False

# Custom model for extracting 1024-D features
efficientnet_model = nn.Sequential(
    base_model.features,
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(1280, 1024),  # Project 1280 → 1024
    nn.ReLU()
)

efficientnet_model.eval().to(device)

Using device: cuda


Sequential(
  (0): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivation(
    

In [13]:
# -------------------------------------------------------------
# Define Image Transformation
# -------------------------------------------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# -------------------------------------------------------------
# Feature Extraction Function
# -------------------------------------------------------------
def extract_features(image_path, transform, device=device):
    """Extract EfficientNet-1024 features from a single image."""
    try:
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            features = efficientnet_model(image).squeeze().cpu().numpy()

        return features
    except Exception as e:
        print(f"Skipping frame: {image_path} due to error: {e}")
        return None


In [18]:
# -------------------------------------------------------------
# Paths to RGB and Label CSV
# -------------------------------------------------------------
rgb_path = "D:/Datasets/Datasets/GTEA/GTEA_NEW/S2_Cheese_C1"
label_csv_path = "D:/Datasets/Datasets/GTEA/GTEA_NEW/Label_CSV/S2_Cheese_C1.csv"
output_csv = "Feature_EfficientNetOnly/Feature_S2_Cheese_C1_GTEA.csv"

# Load label CSV
labels_df = pd.read_csv(label_csv_path)

In [19]:
# -------------------------------------------------------------
# Extract Features and Save to CSV
# -------------------------------------------------------------
S = 1  # Frame sampling
features_list = []

all_frames = sorted(os.listdir(rgb_path))[::S]

for frame in tqdm(all_frames, desc="Extracting RGB Features"):
    rgb_frame_path = os.path.join(rgb_path, frame)

    rgb_features = extract_features(rgb_frame_path, transform, device)

    if rgb_features is not None:
        frame_number = int(frame.split('_')[-1].split('.')[0])
        label_row = labels_df[(labels_df['StartFrame'] <= frame_number) & (labels_df['EndFrame'] >= frame_number)]

        action_label = label_row.iloc[0]['ActionLabel'] if not label_row.empty else 0

        features_list.append([frame, action_label] + rgb_features.tolist())

# -------------------------------------------------------------
# Save Features to CSV
# -------------------------------------------------------------
if len(features_list) == 0:
    raise ValueError("No valid features extracted. Please check the dataset paths and feature extraction function.")

columns = ["Frame", "ActionLabel"] + [f"Feature_{i}" for i in range(1024)]
df = pd.DataFrame(features_list, columns=columns)
os.makedirs(os.path.dirname(output_csv), exist_ok=True)
df.to_csv(output_csv, index=False)

print(f"✅ Feature extraction using EfficientNet-B0 completed! Saved to: {output_csv}")

Extracting RGB Features: 100%|███████████████████████████████████████████████████████| 634/634 [00:14<00:00, 43.76it/s]


✅ Feature extraction using EfficientNet-B0 completed! Saved to: Feature_EfficientNetOnly/Feature_S2_Cheese_C1_GTEA.csv
